## Retrieve parsed collection

In [1]:
import pandas as pd
import joblib
from modules.db import mongo_base

# connection_params = ["twitter", "crowdflower_features"]
# client = mongo_base.connect()
# db_name = connection_params[0]
# connection_params.insert(0, client)

# query = {}
# query["filter"] = {}
# query["projection"] = {}
# query["limit"] = 0
# query["skip"] = 0
# query["no_cursor_timeout"] = True
# cursor = mongo_base.finder(connection_params, query, False)
# df = pd.DataFrame(list(cursor))
# df.head()

In [2]:
crowdflower_persistence = 'data/persistence/crowdflower_features.pkl.compressed'
# joblib.dump(df, crowdflower_persistence, compress=True)

In [34]:
df = joblib.load(crowdflower_persistence)
df.head()

,_id,annotation_label,avg_token_length,bigrams,brown_cluster_ids,char_pentagrams,char_quadgrams,char_trigrams,comment_length,dep_bigrams,...,hashtags,hs_keyword_count,hs_keyword_matches,noun_chunks,text,tokens,trigrams,unknown_words,unknown_words_count,uppercase_token_count
0,58c659be6541913eb7f119dd,not_offensive,4.0,"[Warning penny, penny boards, boards will, wil...","[966, 228, 989, 333, 442, 4618, 602, 19]","[penny, warni, arnin, aggot, board, rning, fag...","[aggo, warn, arni, penn, oard, ards, enny, nin...","[pen, war, nin, mak, enn, you, agg, ggo, fag, ...",9,[warning ROOT NN warning | boards compound NN ...,...,[],1,[faggot],"[{'text': 'warning', 'root': 'warning'}, {'tex...",Warning : penny boards will make you a faggot,"[penny, faggot, warning, boards]","[Warning penny boards, penny boards will, boar...",[],0,0
1,58c659be6541913eb7f119de,hatespeech,4.0,[Fuck dykes],[],[dykes],"[ykes, dyke, fuck]","[fuc, kes, yke, dyk, uck]",2,[dykes nsubj NNP fuck | dykes ROOT VBZ dykes],...,[],0,[],"[{'text': 'fuck', 'root': 'dykes'}]",Fuck dykes,"[fuck, dykes]",[],[],0,0
2,58c659be6541913eb7f119df,hatespeech,7.0,"[user_mention user_mention, user_mention user_...","[124, 3690, 966, 2442, 1684, 86]","[efree, aggot, least, jefre, starr, faggo]","[star, east, tarr, aggo, like, dont, jefr, lea...","[ont, ike, ast, lik, ook, agg, ggo, arr, fag, ...",14,[user_mention compound NN user_mention | user_...,...,[],1,[faggot],"[{'text': 'i', 'root': 'look'}, {'text': 'jefr...",user_mention user_mention user_mention user_me...,"[faggot, user_mention, jefree, like, starr, do...","[user_mention user_mention user_mention, user_...","[user_mention, user_mention, user_mention, use...",6,0
3,58c659be6541913eb7f119e0,hatespeech,4.0,"[user_mention user_mention, user_mention user_...","[228, 228, 1214, 19, 28650, 1014, 981]","[jacki, eeeee, ealou, jealo, ackie, alous, neeee]","[alou, neee, acki, jack, lous, eeee, ealo, cki...","[ack, kie, ous, cki, jea, nee, alo, lou, eal, ...",15,[neeeee nsubj NN user_mention | is dep NN user...,...,[],1,[fag],"[{'text': 'user_mention', 'root': 'neeeee'}, {...",""" user_mention : "" user_mention : user_mention...","[jealous, neeeee, user_mention, jackie, fag]","[user_mention user_mention user_mention, user_...","[user_mention, user_mention, user_mention, nee...",4,0
4,58c659be6541913eb7f119e1,not_offensive,4.0,"[user_mention You, You heard, heard me, me bit...","[858, 26282, 1898, 2485, 148, 12266, 1349, 753...","[talki, texas, nigga, heard, alkin, lking, bit...","[exas, talk, alki, nigg, back, abou, hear, ear...","[itc, alk, lki, nig, kin, tex, hea, you, but, ...",20,[user_mention ROOT NN user_mention | heard nsu...,...,[],1,[bitch],"[{'text': 'user_mention', 'root': 'user_mentio...",user_mention You heard me bitch but any way I'...,"[i'm, user_mention, ass, way, th, bitch, wtf, ...","[user_mention You heard, You heard me, heard m...",[user_mention],1,0


In [5]:
sample = df[0:5]
trigrams = sample['char_trigrams']

In [10]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [21]:
le = LabelEncoder()
le.fit(sample['char_trigrams'][0])
sample_cat = le.transform(sample['char_trigrams'][0])
sample_cat
# list(le.classes_)

array([15, 18, 12, 11,  5, 20,  0,  7,  6,  9, 10,  4, 14,  2,  3,  1, 16,
       19, 13,  8, 17])

In [27]:
ohe = OneHotEncoder()
ohe.fit(sample_cat)
sample_ohe = ohe.transform(sample_cat)
sample_ohe
# ohe.feature_indices_

/home/jherez/Dev/thesis-preprocessing/venv/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/jherez/Dev/thesis-preprocessing/venv/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


<1x21 sparse matrix of type '<class 'numpy.float64'>'
	with 21 stored elements in Compressed Sparse Row format>

In [33]:
sample_dummies = pd.get_dummies(df, columns = ['char_trigrams'])

TypeError: unhashable type: 'list'